# Boston Housing Dataset

## Setup


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"


def get_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y=pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)



# Load datasets
training_set = pd.read_csv("https://raw.githubusercontent.com/aliakh/demo_tensorflow/master/boston_housing/boston_train.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)
test_set = pd.read_csv("https://raw.githubusercontent.com/aliakh/demo_tensorflow/master/boston_housing/boston_test.csv", skipinitialspace=True,
                         skiprows=1, names=COLUMNS)

# Set of 6 examples for which to predict median house values
prediction_set = pd.read_csv("https://raw.githubusercontent.com/aliakh/demo_tensorflow/master/boston_housing/boston_predict.csv", skipinitialspace=True,
                               skiprows=1, names=COLUMNS)

# Feature cols
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

# Build 2 layer fully connected DNN with 10, 10 units respectively.
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                        hidden_units=[10, 10],
                                        model_dir="/tmp/boston_model")

# Train
regressor.train(input_fn=get_input_fn(training_set), steps=5000)

# Evaluate loss over one epoch of test_set.
ev = regressor.evaluate(
      input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

# Print out predictions over a slice of prediction_set.
y = regressor.predict(
      input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False))
# .predict() returns an iterator of dicts; convert to a list and print
# predictions
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4e13f0bb90>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/boston_model', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/boston_model/model.ckpt.
INFO:t

INFO:tensorflow:loss = 6420.175, step = 2201 (0.521 sec)
INFO:tensorflow:global_step/sec: 192.773
INFO:tensorflow:loss = 7028.795, step = 2301 (0.522 sec)
INFO:tensorflow:global_step/sec: 196.393
INFO:tensorflow:loss = 7585.5776, step = 2401 (0.495 sec)
INFO:tensorflow:global_step/sec: 200.11
INFO:tensorflow:loss = 6771.8286, step = 2501 (0.500 sec)
INFO:tensorflow:global_step/sec: 194.318
INFO:tensorflow:loss = 7591.5625, step = 2601 (0.525 sec)
INFO:tensorflow:global_step/sec: 195.21
INFO:tensorflow:loss = 5900.9297, step = 2701 (0.504 sec)
INFO:tensorflow:global_step/sec: 217.662
INFO:tensorflow:loss = 6338.039, step = 2801 (0.465 sec)
INFO:tensorflow:global_step/sec: 194.157
INFO:tensorflow:loss = 5531.953, step = 2901 (0.508 sec)
INFO:tensorflow:global_step/sec: 196.633
INFO:tensorflow:loss = 7458.926, step = 3001 (0.522 sec)
INFO:tensorflow:global_step/sec: 214.013
INFO:tensorflow:loss = 9354.04, step = 3101 (0.456 sec)
INFO:tensorflow:global_step/sec: 199.911
INFO:tensorflow:los

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-10-13:59:39
INFO:tensorflow:Saving dict for global step 5000: average_loss = 21.874023, global_step = 5000, loss = 2187.4023
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/boston_model/model.ckpt-5000
Loss: 2187.402344
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/boston_model/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions: [array([34.19158], dtype=float32), array([19.982697], dtype=float32), array([24.714937], dtype=float32), array([33.642418], dtype=float32), array([15.80559], dtype=float32), array([21.148216], dtype=float32)]
